In [ ]:

from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials
from mat3ra.made.tools.build.nanoribbon import create_nanoribbon, NanoribbonConfiguration
combined_length = 20
material = Material(Material.default_config)
graphene = Material(Materials.get_by_name_first_match("Graphene"))
config = NanoribbonConfiguration(material=graphene, width = 2 * 10, length = combined_length , edge_type = "zigzag")
nanoribbon = create_nanoribbon(config)

In [ ]:
from matplotlib import font_manager

# Get a list of all available fonts
available_fonts = font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

# Print the names of the available fonts
font_names = [font_manager.FontProperties(fname=font).get_name() for font in available_fonts]
unique_font_names = sorted(set(font_names))

print("Available fonts:")
for font_name in unique_font_names:
    print(font_name)

In [ ]:
from mat3ra.made.tools.modify import rotate
from IPython.display import display

import numpy as np
from matplotlib import font_manager
def create_text_grid(digit, grid_size=10):
    """
    Create a grid representation of a single digit.
    Parameters:
        digit (str): A single digit as a string (e.g., '2', '0', etc.).
        grid_size (int): The size of the grid for each digit (default: 10).
    Returns:
        np.ndarray: A binary grid with 1s representing the digit.
    """
    from PIL import Image, ImageDraw, ImageFont

    # Create a blank image with white background
    image = Image.new("1", (grid_size, grid_size), color=1)  # "1" mode for binary image
    draw = ImageDraw.Draw(image)

    #Find the path for a specific fon
    font_name = None
    # Use a bitmap font to disable antialiasing
    if font_name:
        try:
            font_path = font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
            font_path = [fp for fp in font_path if font_name.lower() in fp.lower()]
            if not font_path:
                raise RuntimeError(f"Font '{font_name}' not found.")
            font = ImageFont.truetype(font_path[0], grid_size)
        except IOError:
            raise RuntimeError(f"Failed to load font '{font_name}'. Make sure the font is available.")
    else:
        try:
            font = ImageFont.load_default()
        except IOError:
            raise RuntimeError("Failed to load default font. Make sure PIL's default font is available.")


    # Load a basic font and draw the digit
    draw.text((0, 0), digit, fill=0, font=font)

    # Convert to a binary grid
    grid = np.array(image)

    scale_factor = 10
    # Scale the image for better visualization
    scaled_size = (grid_size * scale_factor, grid_size * scale_factor)
    scaled_image = image.resize(scaled_size, resample=Image.NEAREST)

    # Display the scaled image in the notebook
    print(f"Digit: {digit}")
    display(scaled_image)

    return grid

def create_text_structure(digits, spacing=10, grid_size=10, lattice_param=2.0, x_shift=0):
    """
    Create the atomic coordinates to represent a series of digits.
    Parameters:
        digits (str): The digits to represent (e.g., '2025').
        spacing (int): Spacing between the digits.
        grid_size (int): The size of the grid for each digit.
        lattice_param (float): Lattice parameter for the atomic structure.
    Returns:
        np.ndarray: Array of atomic coordinates for the digits.
    """
    all_coords = []
    x_offset = 0

    for digit in digits:
        # Get the grid for the current digit
        grid = create_text_grid(digit, grid_size)

        # Convert grid positions to coordinates
        x_offset += grid_size * lattice_param + spacing * lattice_param + x_shift
        for y in range(grid.shape[0]):
            for x in range(grid.shape[1]):
                if not grid[y, x]:
                    # Map grid points to 3D coordinates
                    all_coords.append([
                        x * lattice_param + x_offset,
                        y * lattice_param,
                        0.0
                    ])
        # Add spacing between digits

    return np.array(all_coords)



# Parameters
digits = "90"
lattice_param = 2.46  # Distance between atoms
grid_size = 10  # Resolution of the digit
spacing = 0  # Spacing between digits
x_shift = -5

# Generate the coordinates for the digits
coords = create_text_structure(digits, spacing=spacing, grid_size=grid_size, lattice_param=lattice_param, x_shift=x_shift)
# print(coords)

# material = Material(Material.default_config)
# graphene = Material(Materials.get_by_name_first_match("Graphene"))
# config = NanoribbonConfiguration(material=graphene, width = len(digits) * grid_size, length = len(digits) * grid_size * 2 , edge_type = "zigzag")
# material = create_nanoribbon(config)
material = nanoribbon.clone()
# size = lattice_param * grid_size * len(digits) + spacing * lattice_param * (len(digits) - 1)
# material.set_new_lattice_vectors([size, 0, 0], [0, size, 0], [0, 0, 20])


for coord in coords:
    coord_copy = coord
    coord[0] = coord[0] - lattice_param*5
    coord[1] = coord[1] + lattice_param*6
    coord[2] = coord[2] - lattice_param
    material.add_atom("Au", coord, use_cartesian_coordinates=True)
    mirror_coord = coord
    mirror_coord[0] = -coord[0]
    mirror_coord[2] = coord_copy[2] + 2.5*lattice_param
    material.add_atom("Cs", mirror_coord, use_cartesian_coordinates=True)

visualize_materials(material)

In [ ]:
from utils.jupyterlite import set_materials
from utils.visualize import visualize_materials
from mat3ra.made.tools.modify import add_vacuum, translate_to_z_level

material_copy = material.clone()

material_copy = add_vacuum(material_copy, 20, to_bottom=True)
material_copy = translate_to_z_level(material_copy, "center")
material_copy = rotate(material_copy, [1,0,0], -90, rotate_cell=False)

visualize_materials([material_copy], rotation="-180x")
visualize_materials([material_copy], rotation="-90x")

material_copy.name = "90"
set_materials([material_copy])

